In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms

import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
device

'cuda'

In [4]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
mnist_train = datasets.MNIST(root = 'data/', 
                             train = True, 
                             transform = transforms.ToTensor(), 
                             download = True)
mnist_test = datasets.MNIST(root = 'data/', 
                             train = False, 
                             transform = transforms.ToTensor(), 
                             download = True)

In [6]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, 
                                          batch_size = batch_size,
                                         shuffle = True, 
                                         #drop_last = True
                                         )

In [40]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
        out = self.layer3(out)
        #print(out.shape)
        out = out.view(out.size(0), -1)
        #print(out.shape)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [41]:
model = CNN().to(device)

In [42]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)

In [43]:
value = torch.Tensor(1,1,28,28).to(device)
print(model(value))

tensor([[-2.9566e+33, -3.8111e+33, -3.4173e+33, -2.6982e+33, -1.9445e+33,
         -4.6012e+32, -2.9680e+33, -3.3341e+32, -2.5831e+33, -1.8472e+33]],
       device='cuda:0', grad_fn=<AddmmBackward>)


In [44]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [45]:
total_batch = len(data_loader)
total_batch

600

In [46]:
for x, y in  data_loader:
    print(x.shape)
    break

torch.Size([100, 1, 28, 28])


In [ ]:
for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad() # d이것을 뺴면 학습이 아예 안된다. 
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {}] cost = {}'.format(epoch + 1, avg_cost))
print('Learning Finished!')

[Epoch: 1] cost = 0.1552816778421402
[Epoch: 2] cost = 0.04200555011630058
[Epoch: 3] cost = 0.028766216710209846
[Epoch: 4] cost = 0.022731466218829155
[Epoch: 5] cost = 0.01932958886027336
[Epoch: 6] cost = 0.0138260368257761
[Epoch: 7] cost = 0.01320705097168684
[Epoch: 8] cost = 0.009487074799835682
[Epoch: 9] cost = 0.00986579991877079
[Epoch: 10] cost = 0.01016968209296465
[Epoch: 11] cost = 0.007198483217507601


In [ ]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:' , accuracy.item())